Task 3.1: ILP to QUBO

In [2]:
from docplex.mp.model import Model
import dimod
import numpy as np

# ILP for BPP
def ilp_bpp(weights, bin_capacity):
    n = len(weights)
    mdl = Model("BPP")
    x = {(i, j): mdl.binary_var() for i in range(n) for j in range(n)}
    y = {j: mdl.binary_var() for j in range(n)}

    for i in range(n):
        mdl.add_constraint(mdl.sum(x[i, j] for j in range(n)) == 1)

    for j in range(n):
        mdl.add_constraint(mdl.sum(weights[i] * x[i, j] for i in range(n)) <= bin_capacity * y[j])

    mdl.minimize(mdl.sum(y[j] for j in range(n)))
    return mdl

# Transform ILP to QUBO
def ilp_to_qubo(weights, bin_capacity):
    mdl = ilp_bpp(weights, bin_capacity)
    qubo = mdl.to_qubo()
    return qubo

# Test with specific instances
weights_small = [2, 5, 4, 7, 1, 3]
weights_medium = [10, 15, 20, 10, 25, 30]
weights_large = [25, 50, 40, 35, 20, 45, 55]
bin_capacity_small = 10
bin_capacity_medium = 50
bin_capacity_large = 100

qubo_small = ilp_to_qubo(weights_small, bin_capacity_small)
qubo_medium = ilp_to_qubo(weights_medium, bin_capacity_medium)
qubo_large = ilp_to_qubo(weights_large, bin_capacity_large)


ModuleNotFoundError: No module named 'docplex'

Task 3.2: Brute Force Solver

In [3]:
def brute_force_solver(qubo):
    min_energy = float('inf')
    best_solution = None
    for solution in dimod.ExactSolver().sample_qubo(qubo).data(['sample', 'energy']):
        if solution.energy < min_energy:
            min_energy = solution.energy
            best_solution = solution.sample
    return best_solution, min_energy

Task 3.3: Quantum Annealing with D-Wave

In [4]:
from dwave.system import EmbeddingComposite, DWaveSampler

def quantum_annealing(qubo):
    sampler = EmbeddingComposite(DWaveSampler())
    sampleset = sampler.sample_qubo(qubo, num_reads=100)
    return sampleset.first.sample, sampleset.first.energy

Task 3.4: Quantum Variational Ansatz

In [5]:
from qiskit import Aer, QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.algorithms.optimizers import COBYLA
from qiskit.opflow import PauliSumOp, Z, I
from qiskit.utils import QuantumInstance
from qiskit.algorithms import VQE

def variational_ansatz(num_qubits, layers):
    qc = QuantumCircuit(num_qubits)
    for _ in range(layers):
        for q in range(num_qubits):
            theta = Parameter(f'θ{q}')
            qc.rx(theta, q)
            qc.ry(theta, q)
        for q in range(num_qubits - 1):
            qc.cx(q, q + 1)
    return qc

def solve_with_vqe(qubo, ansatz, shots=1024):
    hamiltonian = PauliSumOp.from_list([("Z" * len(qubo), coef) for (_, coef) in qubo.items()])
    vqe = VQE(ansatz, optimizer=COBYLA(), quantum_instance=QuantumInstance(Aer.get_backend('aer_simulator')))
    return vqe.compute_minimum_eigenvalue(hamiltonian).eigenvalue.real


ImportError: cannot import name 'Aer' from 'qiskit' (/home/ronit/.local/lib/python3.12/site-packages/qiskit/__init__.py)

Task 3.5: QAOA Solver

In [6]:
from qiskit.algorithms import QAOA
from qiskit.opflow import PauliSumOp
from qiskit.algorithms.optimizers import SPSA

def qaoa_solver(qubo, p=2):
    hamiltonian = PauliSumOp.from_list([("Z" * len(qubo), coef) for (_, coef) in qubo.items()])
    qaoa = QAOA(optimizer=SPSA(), reps=p, quantum_instance=QuantumInstance(Aer.get_backend('aer_simulator')))
    result = qaoa.compute_minimum_eigenvalue(hamiltonian)
    return result.eigenvalue.real


ModuleNotFoundError: No module named 'qiskit.algorithms'

Task 3.6: Comparison and Analysis

In [7]:
# Results and analysis
bf_solution, bf_energy = brute_force_solver(qubo_small)
qa_solution, qa_energy = quantum_annealing(qubo_small)
vqe_energy = solve_with_vqe(qubo_small, variational_ansatz(len(qubo_small), 2))
qaoa_energy = qaoa_solver(qubo_small)

print("Brute Force:", bf_solution, bf_energy)
print("Quantum Annealing:", qa_solution, qa_energy)
print("VQE Energy:", vqe_energy)
print("QAOA Energy:", qaoa_energy)


NameError: name 'qubo_small' is not defined